# 📊 WP Orders Comprehensive Analytics Dashboard

**Presentation Date**: December 9, 2025

**Purpose**: Consolidated analysis of 26,977 work package orders covering SLA compliance, order status, volume trends, quality metrics, and processing efficiency.

---

## 🔧 Setup & Data Loading

In [ ]:
# Install required packages (for Colab, uncomment if needed)
# !pip install pandas openpyxl plotly kaleido -q

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

pd.set_option('display.max_columns', None)
print("✅ Libraries loaded!")
print(f"📅 Report Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Libraries loaded!
📅 Report Generated: 2025-12-07 19:07


In [ ]:
# File path
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

📁 Using file: C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx


In [ ]:
# Load and prepare data
df = pd.read_excel(filename, engine='openpyxl')
print(f"📊 Loaded: {len(df):,} records")

# Helper to strip timezone from dates
def parse_date(series):
    dt = pd.to_datetime(series, errors='coerce')
    if dt.dt.tz is not None:
        dt = dt.dt.tz_localize(None)
    return dt

# Parse all date columns
df['requested_date'] = parse_date(df.get('WP Requested Delivery Date'))
df['added_date'] = parse_date(df.get('Added Date'))
df['approved_date'] = parse_date(df.get('Approved Date'))
df['acknowledged_date'] = parse_date(df.get('Acknowledgement Date'))
df['submitted_date'] = parse_date(df.get('Submitted Date'))
df['readiness_date'] = parse_date(df.get('WP Readiness Date'))

# Parse quantities
df['quantity'] = pd.to_numeric(df.get('WP Quantity'), errors='coerce').fillna(0)
df['completed'] = pd.to_numeric(df.get('WP Completed Qty'), errors='coerce').fillna(0)

# Constants
TODAY = pd.Timestamp.now().normalize()
TERMINAL = ['Approved', 'Rejected', 'Cancelled']

# Calculate all key metrics
status_col = 'WP Order Status'
df['is_terminal'] = df[status_col].isin(TERMINAL)
df['is_approved'] = df[status_col] == 'Approved'
df['has_target'] = df['requested_date'].notna()
df['is_past_due'] = (df['requested_date'] < TODAY) & df['has_target']
df['is_breached'] = df['is_past_due'] & ~df['is_terminal']
df['days_overdue'] = np.where(df['is_past_due'], (TODAY - df['requested_date']).dt.days, 0)
df['is_at_risk'] = ((df['requested_date'] >= TODAY) & (df['requested_date'] <= TODAY + timedelta(days=7)) & ~df['is_terminal'] & df['has_target'])

# ITD
if 'In-Time Delivery' in df.columns:
    df['itd_clean'] = df['In-Time Delivery'].fillna('').astype(str).str.lower().str.strip()
    df['is_on_time'] = df['itd_clean'].isin(['yes', 'y', '1', 'true'])
    df['itd_known'] = df['itd_clean'].isin(['yes', 'y', '1', 'true', 'no', 'n', '0', 'false'])
else:
    df['itd_known'] = False
    df['is_on_time'] = False

# Cycle time
df['cycle_time'] = (df['approved_date'] - df['added_date']).dt.days
df['time_to_ack'] = (df['acknowledged_date'] - df['added_date']).dt.days
df['time_to_approve'] = (df['approved_date'] - df['submitted_date']).dt.days

print("✅ Data prepared successfully!")

📊 Loaded: 26,977 records
✅ Data prepared successfully!


---

# 📈 SECTION 1: Executive Summary

In [ ]:
# Calculate all KPIs
total_orders = len(df)
active_with_date = (~df['is_terminal'] & df['has_target']).sum()
breached = int(df['is_breached'].sum())
at_risk = int(df['is_at_risk'].sum())
sla_compliance = 100 - (breached / active_with_date * 100) if active_with_date > 0 else 100

total_qty = df['quantity'].sum()
completed_qty = df['completed'].sum()
completion_rate = (completed_qty / total_qty * 100) if total_qty > 0 else 0

approved = int(df['is_approved'].sum())
cancelled = int((df[status_col] == 'Cancelled').sum())
rejected = int((df[status_col] == 'Rejected').sum())
objected = int((df[status_col] == 'Objected').sum())
approval_rate = (approved / (total_orders - cancelled) * 100) if (total_orders - cancelled) > 0 else 0

itd_total = int(df['itd_known'].sum())
on_time = int(df['is_on_time'].sum())
otd_rate = (on_time / itd_total * 100) if itd_total > 0 else 0

avg_cycle = df['cycle_time'].dropna().mean() if df['cycle_time'].notna().any() else 0

# Display Executive Summary
print("="*80)
print("📊 EXECUTIVE SUMMARY")
print("="*80)
print(f"\n📋 Dataset:           {total_orders:,} total orders")
print(f"📅 Period:            Through {TODAY.strftime('%Y-%m-%d')}")
print(f"\n🎯 KEY PERFORMANCE INDICATORS:")
print(f"   • SLA Compliance:    {sla_compliance:.1f}%  {'❌ CRITICAL' if sla_compliance < 50 else '⚠️ NEEDS ATTENTION' if sla_compliance < 85 else '✅'}")
print(f"   • Completion Rate:   {completion_rate:.1f}%  {'✅' if completion_rate >= 70 else '⚠️'}")
print(f"   • Approval Rate:     {approval_rate:.1f}%  {'✅' if approval_rate >= 75 else '⚠️'}")
print(f"   • On-Time Delivery:  {otd_rate:.1f}%  {'✅' if otd_rate >= 90 else '⚠️'}")
print(f"   • Avg Cycle Time:    {avg_cycle:.1f} days")
print(f"\n🚨 IMMEDIATE ATTENTION REQUIRED:")
print(f"   • SLA Breached:      {breached:,} orders ({(breached/active_with_date*100):.1f}% of active)")
print(f"   • At Risk (≤7 days): {at_risk:,} orders")
print(f"   • Rejected:          {rejected:,} orders")
print(f"   • Objected:          {objected:,} orders")
print("="*80)

📊 EXECUTIVE SUMMARY

📋 Dataset:           26,977 total orders
📅 Period:            Through 2025-12-07

🎯 KEY PERFORMANCE INDICATORS:
   • SLA Compliance:    40.2%  ❌ CRITICAL
   • Completion Rate:   73.0%  ✅
   • Approval Rate:     80.6%  ✅
   • On-Time Delivery:  97.2%  ✅
   • Avg Cycle Time:    32.7 days

🚨 IMMEDIATE ATTENTION REQUIRED:
   • SLA Breached:      2,855 orders (59.8% of active)
   • At Risk (≤7 days): 1 orders
   • Rejected:          137 orders
   • Objected:          221 orders


In [ ]:
# Executive KPI Dashboard
fig = make_subplots(
    rows=2, cols=3,
    specs=[[{'type': 'indicator'}]*3, [{'type': 'indicator'}]*3],
    subplot_titles=['Total Orders', 'SLA Compliance', 'Completion Rate', 
                    'Approval Rate', 'On-Time Delivery', 'Avg Cycle Time']
)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    number={'font': {'size': 48, 'color': '#3498DB'}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=sla_compliance,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#FF6B6B' if sla_compliance < 50 else '#FFE66D' if sla_compliance < 85 else '#4ECDC4'},
           'steps': [{'range': [0, 50], 'color': '#FFCDD2'}, {'range': [50, 85], 'color': '#FFF9C4'}, {'range': [85, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=completion_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#2ECC71'},
           'steps': [{'range': [0, 50], 'color': '#FFCDD2'}, {'range': [50, 75], 'color': '#FFF9C4'}, {'range': [75, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=approval_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#9B59B6'},
           'steps': [{'range': [0, 50], 'color': '#FFCDD2'}, {'range': [50, 75], 'color': '#FFF9C4'}, {'range': [75, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=2, col=1)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=otd_rate,
    gauge={'axis': {'range': [0, 100]}, 'bar': {'color': '#E67E22'},
           'steps': [{'range': [0, 80], 'color': '#FFCDD2'}, {'range': [80, 90], 'color': '#FFF9C4'}, {'range': [90, 100], 'color': '#C8E6C9'}]},
    number={'suffix': '%', 'font': {'size': 32}}
), row=2, col=2)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=avg_cycle,
    delta={'reference': 30, 'decreasing': {'color': 'green'}, 'increasing': {'color': 'red'}},
    number={'suffix': ' days', 'font': {'size': 40}}
), row=2, col=3)

fig.update_layout(height=550, title={'text': '📊 Executive KPI Dashboard', 'x': 0.5, 'font': {'size': 24}})
fig.show()

---

# 🚨 SECTION 2: SLA Health Deep Dive

In [ ]:
# SLA Analysis
print("\n🚨 SLA COMPLIANCE ANALYSIS")
print("="*60)
print(f"Active Orders (Non-Terminal): {active_with_date:,}")
print(f"  • Breached:     {breached:,} ({(breached/active_with_date*100):.1f}%)")
print(f"  • At Risk:      {at_risk:,} ({(at_risk/active_with_date*100):.1f}%)")
print(f"  • Compliant:    {active_with_date - breached - at_risk:,} ({100 - (breached/active_with_date*100) - (at_risk/active_with_date*100):.1f}%)")

# Top breached products
product_stats = df[df['has_target'] & ~df['is_terminal']].groupby('Product').agg(
    total_active=('is_breached', 'count'),
    breached=('is_breached', 'sum')
).reset_index()
product_stats['breach_pct'] = (product_stats['breached'] / product_stats['total_active'] * 100).round(1)
product_stats = product_stats[product_stats['total_active'] >= 5].sort_values('breach_pct', ascending=False)

print(f"\n🔴 TOP 10 PRODUCTS BY SLA BREACH RATE:")
for _, row in product_stats.head(10).iterrows():
    print(f"   • {row['Product'][:45]:<45} {row['breach_pct']:>6.1f}% ({int(row['breached'])}/{int(row['total_active'])})")

# Most overdue
overdue = df[df['is_breached'] & (df['days_overdue'] > 0)].nlargest(10, 'days_overdue')
print(f"\n⏰ TOP 10 MOST OVERDUE ORDERS:")
for _, row in overdue.iterrows():
    print(f"   • {row['WP Order ID']:<20} {int(row['days_overdue']):>4} days  ({row['Product'][:30]})")


🚨 SLA COMPLIANCE ANALYSIS
Active Orders (Non-Terminal): 4,778
  • Breached:     2,855 (59.8%)
  • At Risk:      1 (0.0%)
  • Compliant:    1,922 (40.2%)

🔴 TOP 10 PRODUCTS BY SLA BREACH RATE:
   • Automation                                     100.0% (10/10)
   • NA-F/NA-C                                      100.0% (8/8)
   • TMO Mars Tool                                  100.0% (10/10)
   • Session Border Controller                      100.0% (123/123)
   • Subscriber Data  Bulk Migration Tool           100.0% (40/40)
   • NetNumber Titan (ENUM/Ut-Proxy)                100.0% (42/42)
   • RadiSys MRF                                    100.0% (48/48)
   • NIAM, NACM                                     100.0% (5/5)
   • NEDR                                           100.0% (5/5)
   • MMTEL                                          100.0% (22/22)

⏰ TOP 10 MOST OVERDUE ORDERS:
   • WPO00151668           310 days  (SDM)
   • WPO00149837           310 days  (CI)
   • WPO00149836         

In [ ]:
# SLA Compliance visual
labels = ['🚨 Breached', '⚠️ At Risk', '✅ Compliant']
values = [breached, at_risk, active_with_date - breached - at_risk]
colors = ['#FF6B6B', '#FFE66D', '#4ECDC4']

fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    hole=0.5,
    marker_colors=colors,
    textinfo='percent+value',
    textfont_size=16
)])

fig.update_layout(
    title={'text': 'SLA Compliance Status', 'x': 0.5, 'font': {'size': 22}},
    annotations=[{'text': f'{sla_compliance:.0f}%<br>Compliant', 'x': 0.5, 'y': 0.5, 'font_size': 20, 'showarrow': False}],
    height=500,
    showlegend=True,
    legend={'orientation': 'h', 'y': -0.1}
)
fig.show()

In [ ]:
# Top products by breach rate
top_products = product_stats.head(12)

fig = go.Figure(data=[go.Bar(
    y=top_products['Product'],
    x=top_products['breach_pct'],
    orientation='h',
    marker_color='#FF6B6B',
    text=top_products['breach_pct'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside'
)])

fig.update_layout(
    title={'text': 'SLA Breach Rate by Product (Top 12)', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Breach Rate %',
    yaxis={'categoryorder': 'total ascending'},
    height=550,
    margin={'l': 250}
)
fig.add_vline(x=15, line_dash="dash", line_color="green", annotation_text="Target: 15%")
fig.show()

---

# 📦 SECTION 3: Order Status & Pipeline

In [ ]:
# Status breakdown
status_counts = df[status_col].value_counts()

print("\n📦 ORDER STATUS DISTRIBUTION")
print("="*60)
for status, count in status_counts.items():
    pct = count / total_orders * 100
    print(f"   • {status:<35} {count:>7,} ({pct:>5.1f}%)")

print(f"\n📊 KEY METRICS:")
print(f"   • Approval Rate:     {approval_rate:.1f}%")
print(f"   • Rejection Rate:    {(rejected/(approved+rejected+objected)*100):.1f}%")
print(f"   • Objection Rate:    {(objected/(approved+rejected+objected)*100):.1f}%")
print(f"   • Completion Rate:   {completion_rate:.1f}% (qty-based)")


📦 ORDER STATUS DISTRIBUTION
   • Approved                             20,478 ( 75.9%)
   • Acknowledged                          4,330 ( 16.1%)
   • Cancelled                             1,584 (  5.9%)
   • Objected                                221 (  0.8%)
   • Pending Approval                        180 (  0.7%)
   • Rejected                                137 (  0.5%)
   • Pending Acknowledgement                  45 (  0.2%)
   • Waiting for Order Submission              2 (  0.0%)

📊 KEY METRICS:
   • Approval Rate:     80.6%
   • Rejection Rate:    0.7%
   • Objection Rate:    1.1%
   • Completion Rate:   73.0% (qty-based)


In [ ]:
# Status donut chart
colors_map = {
    'Approved': '#4ECDC4', 'Rejected': '#FF6B6B', 'Cancelled': '#95A5A6',
    'Objected': '#FFE66D', 'Pending Acknowledgement': '#3498DB',
    'Acknowledged': '#2ECC71', 'Waiting for Order Submission': '#9B59B6',
    'Pending Approval': '#E67E22'
}

fig = go.Figure(data=[go.Pie(
    labels=status_counts.index,
    values=status_counts.values,
    hole=0.5,
    marker_colors=[colors_map.get(s, '#BDC3C7') for s in status_counts.index],
    textinfo='percent+label',
    textposition='outside'
)])

fig.update_layout(
    title={'text': 'Order Status Distribution', 'x': 0.5, 'font': {'size': 22}},
    annotations=[{'text': f'{total_orders:,}<br>Orders', 'x': 0.5, 'y': 0.5, 'font_size': 18, 'showarrow': False}],
    height=500,
    showlegend=False
)
fig.show()

---

# 📈 SECTION 4: Volume Trends & Forecasting

In [ ]:
# Volume analysis
unique_products = df['Product'].nunique()
unique_customers = df['Customer'].nunique()

print("\n📈 VOLUME & DEMAND SUMMARY")
print("="*60)
print(f"Total Orders:     {total_orders:,}")
print(f"Total Quantity:   {int(total_qty):,}")
print(f"Unique Products:  {unique_products}")
print(f"Unique Customers: {unique_customers}")

# Top products
print(f"\n🏆 TOP 10 PRODUCTS BY ORDER VOLUME:")
product_vol = df['Product'].value_counts().head(10)
for product, count in product_vol.items():
    pct = count / total_orders * 100
    print(f"   • {product[:45]:<45} {count:>6,} ({pct:>5.1f}%)")

# Top customers
print(f"\n👥 TOP 10 CUSTOMERS BY ORDER VOLUME:")
customer_vol = df['Customer'].value_counts().head(10)
for customer, count in customer_vol.items():
    pct = count / total_orders * 100
    print(f"   • {customer[:45]:<45} {count:>6,} ({pct:>5.1f}%)")


📈 VOLUME & DEMAND SUMMARY
Total Orders:     26,977
Total Quantity:   129,620
Unique Products:  219
Unique Customers: 616

🏆 TOP 10 PRODUCTS BY ORDER VOLUME:
   • Nokia Policy Controller (NPC)                  1,541 (  5.7%)
   • Session Border Controller                      1,182 (  4.4%)
   • INFRA                                          1,154 (  4.3%)
   • CSD                                            1,125 (  4.2%)
   • Nokia TAS                                      1,106 (  4.1%)
   • HSS                                            1,035 (  3.8%)
   • NCS                                              945 (  3.5%)
   • NCC                                              902 (  3.3%)
   • CMM (Cloud Mobility Manager)                     837 (  3.1%)
   • CFX-5000                                         814 (  3.0%)

👥 TOP 10 CUSTOMERS BY ORDER VOLUME:
   • T-MOBILE USA, INC.                             2,834 ( 10.5%)
   • BHARTI AIRTEL LTD                                762 (  2.8%)
 

In [ ]:
# Weekly trend
trend_df = df[df['added_date'].notna()].copy()
trend_df['week'] = trend_df['added_date'].dt.to_period('W').dt.start_time
weekly = trend_df.groupby('week').agg(
    orders=('quantity', 'count'),
    breached=('is_breached', 'sum')
).reset_index().tail(12)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=weekly['week'], y=weekly['orders'], name='Orders', marker_color='#3498DB', opacity=0.7),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=weekly['week'], y=weekly['breached'], name='SLA Breached', 
               mode='lines+markers', line=dict(color='#FF6B6B', width=3)),
    secondary_y=True
)

fig.update_layout(
    title={'text': 'Weekly Order Volume & SLA Breaches (Last 12 Weeks)', 'x': 0.5, 'font': {'size': 20}},
    height=400,
    legend={'orientation': 'h', 'y': 1.1}
)
fig.update_yaxes(title_text="Orders", secondary_y=False)
fig.update_yaxes(title_text="SLA Breached", secondary_y=True)
fig.show()

In [ ]:
# Top products & customers side by side
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Top 8 Products by Volume', 'Top 8 Customers by Volume']
)

top_8_products = df['Product'].value_counts().head(8)
fig.add_trace(go.Bar(
    y=top_8_products.index,
    x=top_8_products.values,
    orientation='h',
    marker_color='#3498DB',
    text=top_8_products.values,
    textposition='outside',
    showlegend=False
), row=1, col=1)

top_8_customers = df['Customer'].value_counts().head(8)
fig.add_trace(go.Bar(
    y=top_8_customers.index,
    x=top_8_customers.values,
    orientation='h',
    marker_color='#9B59B6',
    text=top_8_customers.values,
    textposition='outside',
    showlegend=False
), row=1, col=2)

fig.update_layout(height=400, margin={'l': 200})
fig.update_yaxes(categoryorder='total ascending')
fig.show()

---

# ✅ SECTION 5: Quality & Customer Satisfaction

In [ ]:
# Quality metrics
print("\n✅ QUALITY METRICS")
print("="*60)
print(f"On-Time Delivery:   {otd_rate:.1f}% ({on_time:,} of {itd_total:,} tracked)")
print(f"Data Coverage:      {(itd_total/total_orders*100):.1f}%")

print(f"\n🔴 QUALITY ISSUES:")
print(f"   • Rejected Orders:  {rejected:,} ({(rejected/total_orders*100):.2f}%)")
print(f"   • Objected Orders:  {objected:,} ({(objected/total_orders*100):.2f}%)")
print(f"   • Total Issues:     {rejected + objected:,} ({((rejected+objected)/total_orders*100):.2f}%)")

# Rejection reasons
if 'Approved/Rejected Reason' in df.columns:
    reason_df = df[df[status_col].isin(['Rejected', 'Objected'])]
    reasons = reason_df['Approved/Rejected Reason'].dropna()
    reasons = reasons[reasons.str.strip() != '']
    if len(reasons) > 0:
        print(f"\n📋 TOP REJECTION/OBJECTION REASONS:")
        for reason, count in reasons.value_counts().head(7).items():
            print(f"   • {str(reason)[:50]:<50} {count:>5}")


✅ QUALITY METRICS
On-Time Delivery:   97.2% (20,053 of 20,624 tracked)
Data Coverage:      76.5%

🔴 QUALITY ISSUES:
   • Rejected Orders:  137 (0.51%)
   • Objected Orders:  221 (0.82%)
   • Total Issues:     358 (1.33%)

📋 TOP REJECTION/OBJECTION REASONS:
   • Descope of work                                       52
   • WP not delivered                                      43
   • Quality Issue                                         17
   • Resource Crunch, not available in Baseline forecas    16
   • WP delivered by Nokia Internal                         9
   • Competency Issue                                       7
   • Work not completed as per the Acceptance criteria      6


In [ ]:
# Quality gauges
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'indicator'}, {'type': 'indicator'}]],
    subplot_titles=['On-Time Delivery Rate', 'Quality Issue Rate']
)

fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=otd_rate,
    gauge={'axis': {'range': [0, 100]}, 
           'bar': {'color': '#2ECC71'},
           'steps': [{'range': [0, 80], 'color': '#FFCDD2'}, 
                     {'range': [80, 90], 'color': '#FFF9C4'}, 
                     {'range': [90, 100], 'color': '#C8E6C9'}],
           'threshold': {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 90}},
    number={'suffix': '%', 'font': {'size': 36}}
), row=1, col=1)

issue_rate = ((rejected + objected) / total_orders * 100)
fig.add_trace(go.Indicator(
    mode="gauge+number",
    value=issue_rate,
    gauge={'axis': {'range': [0, 10]}, 
           'bar': {'color': '#FF6B6B' if issue_rate > 2 else '#4ECDC4'},
           'steps': [{'range': [0, 1], 'color': '#C8E6C9'}, 
                     {'range': [1, 2], 'color': '#FFF9C4'}, 
                     {'range': [2, 10], 'color': '#FFCDD2'}],
           'threshold': {'line': {'color': 'red', 'width': 4}, 'thickness': 0.75, 'value': 2}},
    number={'suffix': '%', 'font': {'size': 36}}
), row=1, col=2)

fig.update_layout(height=350, title={'text': 'Quality Indicators', 'x': 0.5, 'font': {'size': 20}})
fig.show()

---

# ⏱️ SECTION 6: Processing Efficiency & Cycle Times

In [ ]:
# Processing time stats
ack_mean = df['time_to_ack'].dropna().mean() if df['time_to_ack'].notna().any() else 0
approve_mean = df['time_to_approve'].dropna().mean() if df['time_to_approve'].notna().any() else 0

print("\n⏱️ PROCESSING TIME ANALYSIS")
print("="*60)
print(f"Avg Time to Acknowledge:  {ack_mean:.1f} days")
print(f"Avg Time to Approve:      {approve_mean:.1f} days")
print(f"Avg Total Cycle Time:     {avg_cycle:.1f} days")

# Bottleneck
if ack_mean > 0 and approve_mean > 0:
    bottleneck = "Acknowledge" if ack_mean > approve_mean else "Approve"
    print(f"\n🔍 BOTTLENECK: {bottleneck} stage ({max(ack_mean, approve_mean):.1f} days avg)")

# Slowest products
product_times = df.groupby('Product').agg(
    count=('cycle_time', 'count'),
    avg_cycle=('cycle_time', 'mean')
).reset_index()
product_times = product_times[product_times['count'] >= 10].nlargest(10, 'avg_cycle')

print(f"\n🐌 SLOWEST PRODUCTS BY CYCLE TIME (Top 10):")
for _, row in product_times.iterrows():
    print(f"   • {row['Product'][:40]:<40} {row['avg_cycle']:>6.1f} days (n={int(row['count'])})")


⏱️ PROCESSING TIME ANALYSIS
Avg Time to Acknowledge:  1.9 days
Avg Time to Approve:      3.7 days
Avg Total Cycle Time:     32.7 days

🔍 BOTTLENECK: Approve stage (3.7 days avg)

🐌 SLOWEST PRODUCTS BY CYCLE TIME (Top 10):
   • GSAT (Gold Standard Audit Services)        65.9 days (n=21)
   • Cust SW vMTK Virtual Migration tool        61.9 days (n=27)
   • NOrC                                       56.8 days (n=128)
   • Custom Server                              54.6 days (n=18)
   • Custom Application                         53.3 days (n=21)
   • PAS (Parameterized Audit Services)         53.1 days (n=19)
   • Gold Standard Configuration Manager (GSC   53.0 days (n=23)
   • na                                         52.8 days (n=20)
   • NIAS                                       50.5 days (n=15)
   • CI                                         50.5 days (n=120)


In [ ]:
# Processing time indicators
fig = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'indicator'}]*3],
    subplot_titles=['Avg Acknowledge', 'Avg Approve', 'Avg Total Cycle']
)

fig.add_trace(go.Indicator(
    mode="number",
    value=ack_mean,
    number={'suffix': ' days', 'font': {'size': 32, 'color': '#3498DB'}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=approve_mean,
    number={'suffix': ' days', 'font': {'size': 32, 'color': '#E67E22'}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number+delta",
    value=avg_cycle,
    delta={'reference': 30, 'decreasing': {'color': 'green'}, 'increasing': {'color': 'red'}},
    number={'suffix': ' days', 'font': {'size': 32, 'color': '#9B59B6'}}
), row=1, col=3)

fig.update_layout(height=250, title={'text': 'Average Processing Times', 'x': 0.5, 'font': {'size': 20}})
fig.show()

In [ ]:
# Slowest products chart
if len(product_times) > 0:
    fig = go.Figure(data=[go.Bar(
        y=product_times['Product'],
        x=product_times['avg_cycle'],
        orientation='h',
        marker_color='#9B59B6',
        text=product_times['avg_cycle'].apply(lambda x: f'{x:.1f}d'),
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Slowest Products by Avg Cycle Time (Top 10)', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Average Days',
        yaxis={'categoryorder': 'total ascending'},
        height=450,
        margin={'l': 220}
    )
    fig.show()

---

# 🎯 SECTION 7: Key Findings & Action Items

In [ ]:
print("\n" + "="*80)
print("🎯 KEY FINDINGS & RECOMMENDATIONS")
print("="*80)

print("\n🚨 CRITICAL ISSUES:")
if sla_compliance < 50:
    print(f"   1. SLA compliance at {sla_compliance:.1f}% - IMMEDIATE ACTION REQUIRED")
    print(f"      • {breached:,} orders past due ({(breached/active_with_date*100):.1f}% of active)")
    print(f"      • Focus on top breach products: {', '.join(product_stats.head(3)['Product'].tolist())}")

if breached > 1000:
    print(f"   2. High breach volume: {breached:,} overdue orders")
    print(f"      • Average overdue time: {df[df['is_breached']]['days_overdue'].mean():.1f} days")
    print(f"      • Oldest order: {df[df['is_breached']]['days_overdue'].max():.0f} days overdue")

if avg_cycle > 30:
    print(f"   3. Cycle time above target: {avg_cycle:.1f} days (target: 30 days)")
    if ack_mean > 0 and approve_mean > 0:
        bottleneck = "Acknowledge" if ack_mean > approve_mean else "Approve"
        print(f"      • Bottleneck: {bottleneck} stage")

print("\n✅ POSITIVE HIGHLIGHTS:")
if otd_rate >= 90:
    print(f"   • Excellent on-time delivery: {otd_rate:.1f}%")
if approval_rate >= 80:
    print(f"   • Strong approval rate: {approval_rate:.1f}%")
if completion_rate >= 70:
    print(f"   • Good completion progress: {completion_rate:.1f}%")

print("\n📋 RECOMMENDED ACTIONS:")
print("   1. Address {0:,} breached SLA orders immediately".format(breached))
print("   2. Prioritize {0:,} at-risk orders (due within 7 days)".format(at_risk))
if len(product_stats) > 0:
    worst_product = product_stats.iloc[0]
    print(f"   3. Investigate {worst_product['Product']} ({worst_product['breach_pct']:.1f}% breach rate)")
if avg_cycle > 30:
    print(f"   4. Optimize {bottleneck} process to reduce cycle time")
print(f"   5. Review and resolve {rejected + objected:,} rejected/objected orders")

print("\n📅 NEXT STEPS:")
print("   • Schedule deep-dive session on SLA breaches")
print("   • Assign owners to top 10 overdue orders")
print("   • Review resource allocation for bottleneck processes")
print("   • Set up weekly SLA monitoring cadence")
print("="*80)


🎯 KEY FINDINGS & RECOMMENDATIONS

🚨 CRITICAL ISSUES:
   1. SLA compliance at 40.2% - IMMEDIATE ACTION REQUIRED
      • 2,855 orders past due (59.8% of active)
      • Focus on top breach products: Automation, NA-F/NA-C, TMO Mars Tool
   2. High breach volume: 2,855 overdue orders
      • Average overdue time: 18.0 days
      • Oldest order: 310 days overdue
   3. Cycle time above target: 32.7 days (target: 30 days)
      • Bottleneck: Approve stage

✅ POSITIVE HIGHLIGHTS:
   • Excellent on-time delivery: 97.2%
   • Strong approval rate: 80.6%
   • Good completion progress: 73.0%

📋 RECOMMENDED ACTIONS:
   1. Address 2,855 breached SLA orders immediately
   2. Prioritize 1 at-risk orders (due within 7 days)
   3. Investigate Automation (100.0% breach rate)
   4. Optimize Approve process to reduce cycle time
   5. Review and resolve 358 rejected/objected orders

📅 NEXT STEPS:
   • Schedule deep-dive session on SLA breaches
   • Assign owners to top 10 overdue orders
   • Review resource

---

# 📄 Export Report

In [ ]:
# Export summary to Excel
export_filename = f"comprehensive_dashboard_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    # Executive Summary
    summary_df = pd.DataFrame({
        'KPI': ['Total Orders', 'SLA Compliance', 'Completion Rate', 'Approval Rate', 
                'On-Time Delivery', 'Avg Cycle Time', 'SLA Breached', 'At Risk', 'Rejected', 'Objected'],
        'Value': [f"{total_orders:,}", f"{sla_compliance:.1f}%", f"{completion_rate:.1f}%", 
                  f"{approval_rate:.1f}%", f"{otd_rate:.1f}%", f"{avg_cycle:.1f} days",
                  f"{breached:,}", f"{at_risk:,}", f"{rejected:,}", f"{objected:,}"]
    })
    summary_df.to_excel(writer, sheet_name='Executive Summary', index=False)
    
    # SLA by Product
    product_stats.to_excel(writer, sheet_name='SLA by Product', index=False)
    
    # Status breakdown
    status_df = df[status_col].value_counts().reset_index()
    status_df.columns = ['Status', 'Count']
    status_df['Percentage'] = (status_df['Count'] / total_orders * 100).round(1)
    status_df.to_excel(writer, sheet_name='Status Breakdown', index=False)
    
    # Top overdue
    if len(overdue) > 0:
        overdue_export = overdue[['WP Order ID', 'Product', 'Customer', status_col, 'days_overdue']].copy()
        overdue_export.to_excel(writer, sheet_name='Most Overdue', index=False)

print(f"\n✅ Report exported to: {export_filename}")
# files.download(export_filename)  # Uncomment for Colab


✅ Report exported to: comprehensive_dashboard_20251207_1908.xlsx


---

# 📌 Presentation Notes

## Key Messages for Tomorrow's Call:

1. **SLA Compliance Crisis**: Only 40.2% compliant - immediate action needed on 2,855 breached orders
2. **Quality Remains Strong**: 97.2% on-time delivery and 80.6% approval rate
3. **Volume Concentration**: Top customer (T-Mobile) represents 10.5% of orders
4. **Process Efficiency**: Average 32.7-day cycle time with bottleneck in approval stage
5. **Action Plan**: Focus on top breach products and oldest overdue orders first

## Discussion Points:
- Root causes of SLA breaches in Session Border Controller, Automation products
- Resource allocation to reduce approval bottleneck
- Customer impact mitigation for T-Mobile and top accounts
- Weekly monitoring cadence going forward
